## Churn insurance dataset

Click to run on colab (if you are not already there): [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1Vg3894R2ljhG8zMaf24RUn6mLryXfncS?usp=sharing)


The goal of this notebook is to build an intuition about the provided dataset for Oracle's churn challenge.


In [1]:
!gdown --id 10K5-CAAZb5wIf-WkCTgQo0Xi_xdDFC_h
!curl -L 'https://docs.google.com/spreadsheets/d/11C3RLM1vhL8idoCPwhK6ALRxCFShXesg/export?format=xlsx' -o data-dictionary.xlsx

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=10K5-CAAZb5wIf-WkCTgQo0Xi_xdDFC_h
From (redirected): https://drive.google.com/uc?id=10K5-CAAZb5wIf-WkCTgQo0Xi_xdDFC_h&confirm=t&uuid=ac434833-5aca-44fd-b8a1-46b238fce104
To: /content/ml_churn_final_data.xlsx
100% 199M/199M [00:04<00:00, 45.9MB/s]
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   375    0   375    0     0    953      0 --:--:-- --:--:-- --:--:--   951
100 11645    0 11645    0     0  16655      0 --:--:-- --:--:-- --:--:-- 16655


In [2]:
!sudo apt update -y
!sudo apt install gnumeric
!ssconvert ml_churn_final_data.xlsx dataset.csv

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,070 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,343 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/m

In [4]:
import numpy as np
import pandas as pd # too slow here
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("dataset.csv")

<ipython-input-4-550daa99609c>:7: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("dataset.csv")


### Preliminaries

In [5]:
print(df.head())

    MHNET       SYMB  SYMB_START    SYMB_END PAYMENT_END RENEWAL_SYMB  \
0  202010  103734856  2020/10/12  2021/10/12  2021/01/12   2021/01/12   
1  202010  103731263  2020/10/10  2021/10/10  2021/04/10   2021/04/10   
2  202010  103731762  2020/10/09  2021/10/09  2021/04/09   2021/04/09   
3  202010  103737388  2020/10/17  2021/10/17  2021/04/17   2021/04/17   
4  202010  103739373  2020/10/19  2021/10/19  2021/01/19   2021/01/19   

   CUSTOMER CUST_BIRTH_DATE  GENDER  CUST_ZIP_CODE  ... HAS_RETURNED  \
0   3585539      1977/01/04     1.0          19001  ...            1   
1   3580938      1947/04/17     1.0          74100  ...            1   
2   3581530      1984/11/23     2.0          45500  ...            1   
3   2647509      1954/05/13     1.0          71409  ...            1   
4   3590638      1952/04/07     1.0          12461  ...            1   

  NEW_PREMIUM_PRICE NUM_COMPLAINTS IS_RENEWAL LOSS_RATIO  ASFAL_AMOUNT  \
0             39.18              0          1        0

In [7]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 631479 entries, 0 to 631478
Data columns (total 58 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   MHNET                      631479 non-null  int64  
 1   SYMB                       631479 non-null  int64  
 2   SYMB_START                 631479 non-null  object 
 3   SYMB_END                   631479 non-null  object 
 4   PAYMENT_END                631479 non-null  object 
 5   RENEWAL_SYMB               631479 non-null  object 
 6   CUSTOMER                   631479 non-null  int64  
 7   CUST_BIRTH_DATE            532462 non-null  object 
 8   GENDER                     631146 non-null  float64
 9   CUST_ZIP_CODE              631479 non-null  int64  
 10  AGENT_CD                   631479 non-null  object 
 11  CDIE_AGENT_CD              631479 non-null  object 
 12  CGD_AGENT_CD               631479 non-null  object 
 13  CEMP_AGENT_CD              63

In [8]:
print(df.describe())
#print(df.select(pl.all().exclude(pl.datatypes.Object)).describe())

               MHNET          SYMB      CUSTOMER         GENDER  \
count  631479.000000  6.314790e+05  6.314790e+05  631146.000000   
mean   201992.052255  1.030905e+08  2.859529e+06       1.286179   
std        38.296844  5.292759e+05  5.831978e+05       0.454903   
min    201901.000000  1.011263e+08  4.374800e+04       1.000000   
25%    202006.000000  1.027610e+08  2.536328e+06       1.000000   
50%    202010.000000  1.032333e+08  2.996665e+06       1.000000   
75%    202010.000000  1.035223e+08  3.313094e+06       2.000000   
max    202012.000000  1.037828e+08  3.640400e+06       3.000000   

       CUST_ZIP_CODE        TRPLHR  SYMB_DURATION  NUM_ACTIVE_SYMB  \
count  631479.000000  11747.000000  631479.000000    631479.000000   
mean    41019.584930     37.137227       6.556199         1.245448   
std     23222.677407     14.606907       3.134660         0.680613   
min     10000.000000      3.000000       3.000000         1.000000   
25%     19003.000000     29.000000       3.000

Missing values in each column

In [9]:
print(df.isnull().sum())
#print(df.select([pl.sum(pl.col(column).is_null()).alias(column) for column in df.columns]))

MHNET                             0
SYMB                              0
SYMB_START                        0
SYMB_END                          0
PAYMENT_END                       0
RENEWAL_SYMB                      0
CUSTOMER                          0
CUST_BIRTH_DATE               99017
GENDER                          333
CUST_ZIP_CODE                     0
AGENT_CD                          0
CDIE_AGENT_CD                     0
CGD_AGENT_CD                      0
CEMP_AGENT_CD                     0
CEP3_AGENT_CD                     0
TRPLHR                       619732
SYMB_DURATION                     0
NUM_ACTIVE_SYMB                   0
NUM_ACCIDENTS_INVOLVED            0
NUM_ACCIDENTS_RESP                0
NUM_ACCIDENTS_RESP_FR             0
NUM_ACCIDENTS_RESP_NOT_FR         0
NUM_ACCIDENTS_NOT_RESP            0
LOYALTY_PROGRAM                   0
NUM_VEHICLES                      0
CAR_BRAND                         0
CAR_MODEL                         0
CAR_PROD_YEAR               

Number of unique values in each column

In [10]:
print(df.nunique())
#print(df.select([pl.col(column).n_unique().alias(column) for column in df.columns]))

MHNET                            24
SYMB                         610173
SYMB_START                     1112
SYMB_END                       1119
PAYMENT_END                    1109
RENEWAL_SYMB                   1109
CUSTOMER                     568924
CUST_BIRTH_DATE               36321
GENDER                            3
CUST_ZIP_CODE                  1176
AGENT_CD                       4572
CDIE_AGENT_CD                   159
CGD_AGENT_CD                     35
CEMP_AGENT_CD                     3
CEP3_AGENT_CD                    12
TRPLHR                           31
SYMB_DURATION                     3
NUM_ACTIVE_SYMB                  13
NUM_ACCIDENTS_INVOLVED           16
NUM_ACCIDENTS_RESP               12
NUM_ACCIDENTS_RESP_FR             9
NUM_ACCIDENTS_RESP_NOT_FR         9
NUM_ACCIDENTS_NOT_RESP            7
LOYALTY_PROGRAM                   2
NUM_VEHICLES                      9
CAR_BRAND                      2744
CAR_MODEL                      2744
CAR_PROD_YEAR               